In [2]:
import pandas as pd
import re
import numpy as np
from numpy.linalg import norm
from nltk.util import ngrams

In [4]:
# 后缀标着1的，是用moka-ai/m3e-base
# 后缀标着2的，是用thenlper/gte-large-zh
# 后缀标着3的，是用amu/tao-8k


query_df = pd.read_pickle('query_df_final.pkl') # 导入query数据，这里只有四季酒店的数据，1000条
df = pd.read_pickle('faq_df_final.pkl') # 导入FAQ数据，这个是逸燊做的，大概从1000条里面调了两百多条典型的出来当FAQ

sentiment_common = pd.read_excel('商家回复1.xlsx', sheet_name = '通用性回复') # 崇仁编的回复
sentiment_specific = pd.read_excel('商家回复1.xlsx', sheet_name = '针对性回复（new）') # 同上

In [5]:
# 定义一个余弦相似度，用于计算两个语句之间的相似度
def cosine_similarty(a,b):
    res = np.dot(a,b)/(norm(a)*norm(b))
    return res 

In [6]:
# 创建空 DataFrame，方便之后计算
ans_df = pd.DataFrame()

# 对 1000 条评论，循环执行以下内容
for m in range(0, len(query_df)):

    # 如果评论内容非空，其实这个也没啥用
    if query_df['评论内容_new'].iloc[m] != '':
        
        ### huggingface
        #### 获取相似度分数

        # 获取第 m 个评论的向量
        q = query_df['query_embedding'].iloc[m]
        
        #### 在 FAQ 数据中循环计算相似度
        # df 是 FAQ 数据，所以这里获取 FAQ 向量并计算与评论向量的余弦相似度
        df["similarities"] = df['faq_embedding'].apply(lambda x: cosine_similarty(x[0], q[0]))

        # 对相似度进行排序
        cnt_df = df.sort_values("similarities", ascending=False)

        ##### 设置前 5 个相似的计数
        cntcnt_df = cnt_df.head(5)
        min_score = cntcnt_df['similarities'].iloc[4]  # 获取前 5 个相似中最小的相似度
        max_score = cntcnt_df['similarities'].iloc[0]  # 获取前 5 个相似中最大的相似度

        # 对分类结果进行分组并计数
        group_df = cntcnt_df.groupby('分类结果')['faq_new'].count().reset_index(name='cnt')
        group_df = group_df.sort_values("cnt", ascending=False)
        res = group_df.head(1)['分类结果'].iloc[0]  # 获取出现次数最多的分类结果
        ratio_score = group_df.head(1)['cnt'].iloc[0] / 5  # 计算该分类结果占前 5 个相似 FAQ 的比例

        # 将结果存入 ans_df
        ans_df.at[m, 'review_no'] = query_df['评论序号'].iloc[m]
        ans_df.at[m, 'query_long'] = query_df['评论内容'].iloc[m]
        ans_df.at[m, 'ngram'] = 1
        ans_df.at[m, 'query'] = query_df['评论内容_new'].iloc[m]
        ans_df.at[m, 'predict'] = res    
        ans_df.at[m, 'minscore'] = min_score  
        ans_df.at[m, 'maxscore'] = max_score          
        ans_df.at[m, 'ratio'] = ratio_score

        



In [7]:
# 对结果进行排序，按照 review_no、query_long 和 ngram 排序
ans_df = ans_df.sort_values(by=['review_no','query_long','ngram'], ascending = [True, False, True])

# 显示前 10 行结果
print(ans_df.head(10))

# 将结果保存到 Excel 文件中
ans_df.to_excel('result_unfiltered.xlsx')



    review_no                                         query_long  ngram  \
0         1.0                    酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。    1.0   
1         1.0                    酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。    1.0   
2         1.0                    酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。    1.0   
3         1.0                    酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。    1.0   
4         1.0                    酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。    1.0   
6         2.0                                         前台接待的M美女很好    1.0   
7         3.0                                    位置好，房间现代、干净、漂亮！    1.0   
8         3.0                                    位置好，房间现代、干净、漂亮！    1.0   
10        4.0  前台的态度很好，升级了行政房 房间设备非常人性化，卫生间是自动感应的,房间温度调试的很舒适，...    1.0   
11        4.0  前台的态度很好，升级了行政房 房间设备非常人性化，卫生间是自动感应的,房间温度调试的很舒适，...    1.0   

               query       predict  minscore  maxscore  ratio  
0             酒店挺不错的       服务-态度-好  0.885521  0.910004    0.8  
1             房间干净卫生    房间-清洁程度-干净  0.899958  

In [8]:
# 筛选出满足以下条件的记录：
# best_min: 0.76
# best_max: 0.88
# best_ratio: 0.7
filtered_df = ans_df[(ans_df['minscore'] >= 0.76) & (ans_df['maxscore'] >= 0.88) & (ans_df['ratio'] >= 0.7)]


In [9]:
# 对 filtered_df 按以下字段排序：
# 1. review_no（评论序号），升序
# 2. query_long（原始评论内容），降序
# 3. predict（预测的分类结果），降序
# 4. ratio（比例得分），升序
# 5. maxscore（最大相似度），升序
# 然后对每个组（由 review_no、query_long 和 predict 组合）生成排名（rank），
# 排名从 1 开始，且在每个组内排名是按上面的排序条件倒序排列的。
filtered_df['rank'] = filtered_df.sort_values(by=['review_no','query_long','predict','ratio','maxscore'],
                                               ascending = [True, False, False, True, True]).groupby(['review_no','query_long','predict']).cumcount(ascending=False) + 1
filtered_df = filtered_df[filtered_df['rank']==1]

/var/folders/0h/yp5_0sv92pb6b5x_2748bs380000gn/T/ipykernel_43032/2216456995.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['rank'] = filtered_df.sort_values(by=['review_no','query_long','predict','ratio','maxscore'],


In [10]:
filtered_df

,review_no,query_long,ngram,query,predict,minscore,maxscore,ratio,rank
0,1.0,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,1.0,酒店挺不错的,服务-态度-好,0.885521,0.910004,0.8,1
1,1.0,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,1.0,房间干净卫生,房间-清洁程度-干净,0.899958,0.900690,1.0,1
3,1.0,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,1.0,床品舒适,房间-舒适度-舒适,0.898947,0.919234,1.0,1
6,2.0,前台接待的M美女很好,1.0,前台接待的M美女很好,服务-态度-好,0.824315,0.911532,0.8,1
8,3.0,位置好，房间现代、干净、漂亮！,1.0,房间现代、干净、漂亮,房间-设施-好,0.879841,0.886175,1.0,1
...,...,...,...,...,...,...,...,...,...
4451,997.0,服务很好，还给孩子加了一张小床，给升级了房间，房间很大。客房服务很赞地理位置优越旁边吃的太多了,1.0,服务很好,服务-态度-好,0.911099,0.911099,1.0,1
4454,997.0,服务很好，还给孩子加了一张小床，给升级了房间，房间很大。客房服务很赞地理位置优越旁边吃的太多了,1.0,房间很大,房间-大小-偏大,0.881123,0.900854,0.8,1
4457,998.0,性价比一般，服务还可以,1.0,服务还可以,服务-态度-好,0.886392,0.896914,1.0,1
4458,999.0,酒店位置很不错，地铁就在楼下，自驾车到酒店，换乘地铁出行都很方便...,1.0,酒店位置很不错,位置-便利性-交通-发达,0.899407,0.916327,0.8,1


In [11]:
# 对 filtered_df 按照 review_no 和 query_long 进行排序
# 其中 review_no（评论序号）是升序，query_long（原始评论内容）也是升序
filtered_df.sort_values(by=['review_no', 'query_long']).to_excel('result_filtered.xlsx')


In [12]:
# 对 filtered_df 按照 'predict' 进行分组，并统计每个 'predict' 分类结果中的唯一 'review_no'（评论序号）的数量
result_df = filtered_df.groupby('predict')['review_no'].nunique().reset_index(name='review_cnt')

# 按照 'review_cnt'（评论数量）降序排序
result_df = result_df.sort_values(['review_cnt'], ascending=False)


In [ ]:
# 逸燊手动调整了商家回复，因此这段函数没有意义

'''
# 使用前向填充方法填充 'level 1' 列中的缺失值
sentiment_specific['level 1'] = sentiment_specific['level 1'].ffill()

# 使用前向填充方法对每个 'level 1' 组中的 'level 2' 列进行填充
sentiment_specific['level 2'] = sentiment_specific.groupby(['level 1'])['level 2'].ffill()

# 使用前向填充方法对每个 'level 1' 和 'level 2' 组中的 'level 3' 列进行填充
sentiment_specific['level 3'] = sentiment_specific.groupby(['level 1', 'level 2'])['level 3'].ffill()

# 使用前向填充方法对每个 'level 1'、'level 2' 和 'level 3' 组中的 'level 4' 列进行填充
sentiment_specific['level 4'] = sentiment_specific.groupby(['level 1', 'level 2', 'level 3'])['level 4'].ffill()

# 定义一个函数，将多级别列连接成一个字符串
def concat(row):
    if pd.isnull(row['level 2']):
        res = row['level 1']
    elif pd.isnull(row['level 3']):
        res = row['level 1'] + '-' + row['level 2']
    elif pd.isnull(row['level 4']):
        res = row['level 1'] + '-' + row['level 2'] + '-' + row['level 3']
    else:
        res = row['level 1'] + '-' + row['level 2'] + '-' + row['level 3'] + '-' + row['level 4']
        
    return res

# 应用 concat 函数，将多级别列连接成一个新的 'cat' 列
sentiment_specific['cat'] = sentiment_specific.apply(concat, axis=1)
'''



In [24]:
sentiment_specific['temp'] = sentiment_specific['preface'] +sentiment_specific['正文']

In [25]:
sentiment_specific

,分类结果,敬语,preface,正文,epilogue,temp
0,房间-大小-偏大,尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。,针对房间的大小问题，很高兴听到您对房间的宽敞感到满意。,如果您有任何需要或需要额外的设施与服务，请随时联系我们，我们随时为您提供帮助。,期待未来能有机会为您提供更满意的服务，欢迎您再次光临。,针对房间的大小问题，很高兴听到您对房间的宽敞感到满意。如果您有任何需要或需要额外的设施与服务...
1,房间-大小-合适,尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。,针对房间的大小问题，我们很高兴房间的大小符合您的期待。,如果您有任何需要或需要额外的设施与服务，请随时联系我们，我们随时为您提供帮助。,期待未来能有机会为您提供更满意的服务，欢迎您再次光临。,针对房间的大小问题，我们很高兴房间的大小符合您的期待。如果您有任何需要或需要额外的设施与服务...
2,房间-大小-偏小,尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。,对于您觉得房间偏小的问题，我们深感抱歉。,您的舒适是我们的首要任务，请告知我们具体的改进建议。我们将尽全力提升您的入住体验。,期待未来能有机会为您提供更满意的服务，欢迎您再次光临。,对于您觉得房间偏小的问题，我们深感抱歉。您的舒适是我们的首要任务，请告知我们具体的改进建议。...
3,房间-布局-合理,尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。,我们很高兴听到您对房间的布局感到满意。,您的肯定是我们不断追求卓越的动力。,期待未来能有机会为您提供更满意的服务，欢迎您再次光临。,我们很高兴听到您对房间的布局感到满意。您的肯定是我们不断追求卓越的动力。
4,房间-布局-不合理,尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。,对于您觉得房间布局不合理的问题，我们深表歉意。,您的宝贵反馈，这将帮助我们不断改进。请您告知具体的改进建议，以便我们能够更好地满足您的需求，...,期待未来能有机会为您提供更满意的服务，欢迎您再次光临。,对于您觉得房间布局不合理的问题，我们深表歉意。您的宝贵反馈，这将帮助我们不断改进。请您告知具...
5,房间-清洁程度-干净,尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。,我们很高兴听到您对房间的清洁程度感到满意。,确保每一位宾客在干净、舒适的环境中入住是我们的首要任务。,期待未来能有机会为您提供更满意的服务，欢迎您再次光临。,我们很高兴听到您对房间的清洁程度感到满意。确保每一位宾客在干净、舒适的环境中入住是我们的首要任务。
6,房间-清洁程度-不干净,尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。,对于房间不够干净这个问题，我们真诚地向您致歉。,保持房间的清洁是我们的责任，我们对此感到十分遗憾。我们将立即采取措施解决这个问题，并确保今后...,期待未来能有机会为您提供更满意的服务，欢迎您再次光临。,对于房间不够干净这个问题，我们真诚地向您致歉。保持房间的清洁是我们的责任，我们对此感到十分遗...
7,房间-舒适度-舒适,尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。,听到您对房间舒适度的称赞，我们感到非常高兴。,在四季酒店，我们努力确保每位客人都能感受到家一般的舒适。您的满意是我们最大的动力。,期待未来能有机会为您提供更满意的服务，欢迎您再次光临。,听到您对房间舒适度的称赞，我们感到非常高兴。在四季酒店，我们努力确保每位客人都能感受到家一般...
8,房间-舒适度-不适,尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。,对于您认为房间不够舒适这个问题，我们深感抱歉。,您的舒适和满意对我们至关重要，我们会立即采取措施，调查并改善这一情况。同时，如果您有任何其他...,期待未来能有机会为您提供更满意的服务，欢迎您再次光临。,对于您认为房间不够舒适这个问题，我们深感抱歉。您的舒适和满意对我们至关重要，我们会立即采取措...
9,房间-隔音水平-安静,尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。,感谢您对于房间隔音质量的认可。,确保您在安静的环境中休息是我们的责任和承诺。您的舒适与安宁是我们不懈追求的目标。,期待未来能有机会为您提供更满意的服务，欢迎您再次光临。,感谢您对于房间隔音质量的认可。确保您在安静的环境中休息是我们的责任和承诺。您的舒适与安宁是我...


In [16]:
# 在 filtered_df 当中，我们把1000条评论，拆成了很多词句，然后为每个词句评估了所属的FAQ，这里是把语句的FAQ按照评论内容的序号映射回评论内容，这样一个评论内容就有多个FAQ了 
# 对 filtered_df 按照 'review_no' 进行分组，并获取每个 'review_no' 对应的唯一 'predict' 值
# 结果 DataFrame 的每一行包含一个评论序号和对应的唯一预测结果数组
each_df = filtered_df.groupby('review_no')['predict'].unique().reset_index(name='prediction_result')

# 显示结果 DataFrame each_df
each_df


,review_no,prediction_result
0,1.0,"[服务-态度-好, 房间-清洁程度-干净, 房间-舒适度-舒适]"
1,2.0,[服务-态度-好]
2,3.0,[房间-设施-好]
3,4.0,"[服务-态度-好, 位置-便利性-交通-发达]"
4,5.0,[服务-态度-好]
...,...,...
581,996.0,[服务-态度-好]
582,997.0,"[服务-态度-好, 房间-大小-偏大]"
583,998.0,[服务-态度-好]
584,999.0,[位置-便利性-交通-发达]


In [29]:
# 定义一个函数，根据每行的预测结果生成相应的回复， 本来KN这里想判断一下的，如果差评到一定地步才用针对性回复，但是我们给暴力解开了，没使用分数，直接给每个分类赋予了一个回答
def match(row):
    # 初始化空字符串，用于存储负面评论
    sentence_neg = ''
    # 初始化一个空列表，用于存储评分
    score_list = []
    res = ''
    # 遍历每个预测结果
    for k in range(0, len(row['prediction_result'])):
        # 获取当前预测结果的分类
        cat = row['prediction_result'][k]
        # 根据分类从 sentiment_specific 数据框中过滤对应的行
        sentiment_df = sentiment_specific[sentiment_specific['分类结果'] == cat]

        # 如果过滤后的 DataFrame 只有一行
        if len(sentiment_df) == 1:
            # 如果该行的 '评论' 列不为空
            if pd.isnull(sentiment_df['temp'].iloc[0]) == False:
                # 将该行的 '评论' 添加到 sentence_neg 中
                sentence_neg = sentence_neg + sentiment_df['temp'].iloc[0]
                # 评分为 -2，表示有针对性回复
                score = -2
                # 将评分添加到 score_list 中
                score_list.append(score)
            else:
                # 根据 '回复类型' 进行评分
                if sentiment_df['回复类型'].iloc[0] == '通用好评':
                    score = 1
                elif sentiment_df['回复类型'].iloc[0] == '通用中性':
                    score = 2
                elif sentiment_df['回复类型'].iloc[0] == '通用差评':
                    score = -1
                
                # 将评分添加到 score_list 中
                score_list.append(score)
    
    # 如果 score_list 不为空
    if len(score_list) > 0:
        # 根据 score_list 中的最小评分生成对应的回复
        if min(score_list) == -2:
            res = sentence_neg
        elif min(score_list) == -1:
            res = sentiment_common['差评'].iloc[0]
        elif min(score_list) == 1:
            res = (sentiment_common['好评'].iloc[0] +
                   sentiment_common['促销'].iloc[0] +
                   sentiment_common['结语'].iloc[0])
        elif min(score_list) == 2:
            res = (sentiment_common['中性评'].iloc[0] +
                   sentiment_common['推广'].iloc[0] +
                   sentiment_common['结语'].iloc[0])
                
    return res

# 对 each_df 应用 match 函数，生成对应的回复并存储在 'response' 列中
each_df['response'] = each_df.apply(match, axis=1)
each_df['response'] = '尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。'+ each_df.apply(match, axis=1) + '期待未来能有机会为您提供更满意的服务，欢迎您再次光临。'

In [30]:
each_df

,review_no,prediction_result,response
0,1.0,"[服务-态度-好, 房间-清洁程度-干净, 房间-舒适度-舒适]",尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。非常感谢您对我们服务态度的肯定。我们...
1,2.0,[服务-态度-好],尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。非常感谢您对我们服务态度的肯定。我们...
2,3.0,[房间-设施-好],尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。感谢您对我们房间设施的肯定！我们会继...
3,4.0,"[服务-态度-好, 位置-便利性-交通-发达]",尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。非常感谢您对我们服务态度的肯定。我们...
4,5.0,[服务-态度-好],尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。非常感谢您对我们服务态度的肯定。我们...
...,...,...,...
581,996.0,[服务-态度-好],尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。非常感谢您对我们服务态度的肯定。我们...
582,997.0,"[服务-态度-好, 房间-大小-偏大]",尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。非常感谢您对我们服务态度的肯定。我们...
583,998.0,[服务-态度-好],尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。非常感谢您对我们服务态度的肯定。我们...
584,999.0,[位置-便利性-交通-发达],尊敬的宾客，感谢您选择我们四季酒店，并分享您的入住体验。我们很高兴您对周边交通的便利性感到满...


In [31]:
each_df.to_excel('response_sentiment.xlsx') 